# MDAnalysis and Pilot-In-Memory


The main performance bottleneck of the current MDAnalysis implementation is the construction of the graph using NetworkX taking ~78% of the overall runtime.


**Beckstein Profiling:**

    47        10           33      3.3      0.0      if adj is None:
    48        10        66544   6654.4      0.0          x = atoms.positions

    54        10     58689221 5868922.1     18.8          adj = (MDAnalysis.core.parallel.distances.distance_array(x, x, box=box) < cutoff)
    
    58        10           78      7.8      0.0      adjk = adj if Nmax is None else adj[:Nmax, :Nmax] 
    59        10    243009076 24300907.6   77.9      graph = nx.Graph(adjk)
    60        10      4346636 434663.6      1.4      subgraphs = nx.connected_components(graph)
    61        49        83597   1706.1      0.0      indices = [np.sort(g) for g in subgraphs]
    62        49      5694698 116218.3      1.8      return [atoms[group].residues for group in indices]



## 1. LeafletFinder NetworkX Implementation Profiling

see https://code.google.com/p/mdanalysis/

Profile default implementation based on [NetworkX](https://networkx.github.io/)

In [ ]:
FILENAME="../data/mdanalysis/small/graph_edges_95_215.csv"
!head -n 5 {FILENAME}

In [ ]:
%matplotlib inline
%time
import networkx as NX
import time
import datetime
import sys

start = time.time()
nxg = NX.read_edgelist(FILENAME, delimiter=",")
end_read = time.time()
NX.draw(nxg, pos=NX.spring_layout(nxg))

In [ ]:
import matplotlib.pyplot as plt
degree_sequence=sorted(NX.degree(nxg).values(),reverse=True) # degree sequence
print "Degree sequence", degree_sequence
print "Length: %d" % len(degree_sequence)

dmax=max(degree_sequence)

plt.loglog(degree_sequence,'b-',marker='o')
plt.title("Degree Histogram")
plt.ylabel("Degree")
plt.xlabel("Node")

In [ ]:
start = time.time()
components = NX.connected_components(nxg)
end_created = time.time()
count = 0
for component in components:
    print str(sorted(component))
    count = count + 1
end_connected = time.time()
print ("Number of Nodes: " + str(NX.number_of_nodes(nxg)))
print ("Number of Edges: " + str(NX.number_of_edges(nxg)))
print ("Connected Components: " + str(count))
print ("Runtime: " + str((end_connected-start)))
print ("Graph Creation Runtime: " + str((end_created-start)))
print ("Connected Components Runtime: " + str((end_connected - end_created)))

In [ ]:
import os
from pilot_hadoop import PilotComputeService
from IPython.display import HTML

os.environ["SAGA_VERBOSE"]="100"

## 2. Pilot-Spark and Pilot-InMemory Implementation

Setup Spark cluster on local machine or HPC resource. Execute **either** 2.1.1 or 2.1.2

### 2.1.1 Start Spark Cluster using Pilot-Spark (Stampede)

see https://github.com/drelu/saga-hadoop

In [ ]:
pilot_compute_description = {
                            "resource_url":"fork://localhost",
                            "number_cores": 1,
                            "cores_per_node":1,
                            "type":"spark"
                            }
pilot = PilotComputeService.create_pilot(pilot_compute_description);

# print out details of Pilot-Spark
details = pilot.get_details()
HTML("<a target='blank' href='%s'>Spark Web UI</a>"%details["web_ui_url"])

### 2.1.2 Start Spark Cluster inside YARN (Chameleon)

In [1]:
%run util/init_spark.py

NUMBER_EXECUTORS=4

from pilot_hadoop import PilotComputeService as PilotSparkComputeService

pilotcompute_description = {
    "service_url": "yarn-client://yarn.radical-cybertools.org",
    "number_of_processes": NUMBER_EXECUTORS,
    "physical_memory_per_process": "3G" 
}

print "SPARK HOME: %s"%os.environ["SPARK_HOME"]
print "PYTHONPATH: %s"%os.environ["PYTHONPATH"]

start = time.time()
pilot_spark = PilotSparkComputeService.create_pilot(pilotcompute_description=pilotcompute_description)
sc = pilot_spark.get_spark_context()
print "Spark Startup, %.2f"%(time.time()-start)

SPARK Home: /usr/hdp/2.3.2.0-2950/spark-1.5.2-bin-hadoop2.6
SPARK HOME: /usr/hdp/2.3.2.0-2950/spark-1.5.2-bin-hadoop2.6
PYTHONPATH: /usr/hdp/2.3.2.0-2950/spark-1.5.2-bin-hadoop2.6/python:/usr/hdp/2.3.2.0-2950/spark-1.5.2-bin-hadoop2.6/python/lib/py4j-0.8.2.1-src.zip
Spark Startup, 10.44


In [2]:
rdd=sc.parallelize(range(2))

In [3]:
import os
import subprocess

rdd.map(lambda a: subprocess.check_output('python --version', shell=True, stderr=subprocess.STDOUT)).collect()

['Python 2.7.5\n', 'Python 2.7.5\n']

## 2.3 Distance Computation

In [2]:
import numpy as np
coord = np.loadtxt("vesicle_1_5M_373_stride1000.xtc_145746Atoms.np_txt", dtype='float32')

### Optimized

In [ ]:
coord_broadcast = sc.broadcast(coord[:10])

In [ ]:
coord_all = coord_broadcast.value

In [5]:
coord_all[1:5]

array([[ 453.69998169,  525.39996338,   53.5       ],
       [ 448.5       ,  524.39996338,   49.5       ],
       [ 434.29998779,  508.99996948,   56.59999847],
       [ 459.29998779,  516.79998779,   55.79999924]], dtype=float32)

In [6]:
part_rdd=sc.parallelize(range(len(coord_broadcast.value)), NUMBER_EXECUTORS)
part_rdd.cache()

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:423

The following code wraps the MDAnalysis functions into Spark code that is executed in a data-parallel way either on an individual or a batch of points (1-D partitioning)

In [7]:
from MDAnalysis.core.distances import distance_array, self_distance_array
from MDAnalysis.analysis.distances import contact_matrix
import scipy.sparse
from scipy.spatial.distance import cdist
import sys
import gc


cutoff = 15.0

################################################################################
# Process batch of points (a partition of the RDD)

def get_edges_partition(adjacency_matrix, cutoff=15.0):
    it = np.nditer(adjacency_matrix, flags=['multi_index'])
    edge_list = []
    while not it.finished:
        value = it[0]
        if cutoff < value:
            # only connect 1 undirectional edge, e.g. <0,1>, but not <1,0>'
            if it.multi_index[0]<=it.multi_index[1]:
                edge_list.append((it.multi_index[0], it.multi_index[1]))
                #print "%d <%s>" % (it[0], it.multi_index),
        it.iternext()
    return edge_list

def compute_distance_partition(iterator):
    """Partition points in 1-D"""
    min_value=sys.maxint
    max_value=-sys.maxint-1
    for i in iterator:
        if i < min_value:
            min_value = i
        if i > max_value:
            max_value = i
    
    # 2-D Partitioning
    coord_all = coord_broadcast.value
    coord_part = coord_all[min_value:max_value]
    #print "**All**"
    #print str(coord_all)
    #print "**Part**"
    #print str(coord_part)
    #adj=contact_matrix(coord_part, returntype="sparse")
    #adj = distance_array(coord_part, coord_all, box=None)
    adj = cdist(coord_part, coord_all)
    #print "**scipy.spatial.distance.cdist**"
    #print(adj)
    #adj2 = distance_array(coord_part, coord_all, box=None)
    #print "**MDAnalysis**"
    #print(adj2)
    
    edge_list = get_edges_partition(adj)
    del coord_part
    del coord_all
    del adj
    gc.collect()
    return edge_list


################################################################################
# Process one point at a time
def get_edges_point(point_index, adjacency_matrix, cutoff=15.0):
    edge_list = []
    for index, i in np.ndenumerate(adjacency_matrix):
        #print ("Index: %d, Value: %d"%(index[i], i))
        #if point_index<=index[1] and i<cutoff:
        if i==True and point_index<=index[1]:
            # Attention we only compute the upper half of the adjacency matrix
            # thus we need to offset the target edge vertice by point_index
            edge_list.append((point_index, point_index+index[1]))
    #del adjacency_matrix
    return edge_list

def compute_distance(point_index):
    # 1-D Partitioning
    coord_all = coord_broadcast.value
    coord_part = coord_all[point_index-1:point_index]
    #adj = (distance_array(coord_part, coord_all[point_index:], box=None) < cutoff)
    adj = (cdist(coord_part, coord_all) < cutoff)
    #adj = cdist(coord_part, coord_all)
    edge_list = get_edges_point(point_index, adj)
    del adj
    #del coord_part
    #del coord_all
    #gc.collect()
    return edge_list

/opt/anaconda/lib/python2.7/site-packages/MDAnalysis/core/distances.py:9: DeprecationWarning: distances has moved to MDAnalysis.lib.distances and will be removed from here in release 1.0
  DeprecationWarning)


In [8]:
start = time.time()
edges_list=part_rdd.map(compute_distance).flatMap(lambda a: a).collect()
#edges_list_spark=part_rdd.mapPartitions(compute_distance_partition).collect()
print str(len(edges_list))
print "ComputeDistanceSpark, %d, %d, %.2f"%(len(coord_all), NUMBER_EXECUTORS, (time.time()-start))


115093
ComputeDistanceSpark, 20000, 96, 378.80


In [8]:
#start = time.time()
#edges_list_local = compute_distance_partition(iter(range(20000)))
#print str(len(edges_list))
#print "ComputeDistanceLocal, %d, %.2f"%(len(coord_all),(time.time()-start))

for i in range(1):
    start = time.time()
    edges_list=part_rdd.map(compute_distance).flatMap(lambda a: a).collect()
    #edges_list_spark=part_rdd.mapPartitions(compute_distance_partition).collect()
    print str(len(edges_list))
    print "ComputeDistanceSpark, %d, %d, %.2f"%(len(coord_all), NUMBER_EXECUTORS, (time.time()-start))
    del edges_list

115093
ComputeDistanceSpark, 20000, 96, 114.90


## Unoptimized using cartesian product

Not good for sparse result data. Only usable on a very small sample:

    sample=row_rdd.sample(False, 0.01, 81)
    sample.count()


In [3]:
import pyspark.mllib.linalg.distributed
coord_matrix=pyspark.mllib.linalg.distributed.RowMatrix(sc.parallelize(coord[:200], 4))
row_rdd=coord_matrix.rows
row_rdd.cache()

MapPartitionsRDD[3] at mapPartitions at PythonMLLibAPI.scala:1480

In [4]:
row_rdd.count()

200

In [14]:
coord = np.loadtxt("md_centered.xtc_95Atoms.np_txt", dtype='float32')
#coord_str = np.array2string(coord, separator=",")

In [17]:
coord

array([[ 36.83100128,  20.13500023,  37.15499878],
       [ 46.68199921,  66.16099548,  63.26799774],
       [ 67.522995  ,  68.6230011 ,  54.61399841],
       [ 40.35400009,  14.70999908,  42.97800064],
       [ 42.58300018,  36.86699677,  11.01300049],
       [ 22.73099899,  54.88199615,  39.13800049],
       [ 69.51100159,  34.54999924,  27.00699997],
       [ 49.21099854,  27.49599838,  60.3809967 ],
       [ 49.60199738,  20.21199989,   6.79399967],
       [ 30.53499985,   4.55800009,   7.03599977],
       [  9.27799988,   8.9829998 ,  38.02500153],
       [  8.0369997 ,  66.09799957,  39.86299896],
       [ 55.26000214,   4.08699989,  67.3999939 ],
       [ 30.85899734,  59.57500076,  10.19099998],
       [ 26.04100037,  13.67999935,  44.57299805],
       [ 51.84799576,  22.13299942,  33.64899826],
       [  7.06699944,  25.31699944,  59.99300003],
       [ 23.29599762,  33.74100113,  74.61499786],
       [ 64.36899567,  68.68099976,  30.96899986],
       [ 18.94499969,  39.68700

In [20]:
coord.shape[0]

95

In [32]:
coord_str=[]
for i in range(len(coord)):
    coord_str.append(str(coord[i][0]) +","+ str(coord[i][1]) +","+ str(coord[i][2]))

array([[ 36.83100128,  20.13500023,  37.15499878],
       [ 46.68199921,  66.16100311,  63.26800156],
       [ 67.52300262,  68.6230011 ,  54.61399841],
       [ 40.35400009,  14.71000004,  42.97800064],
       [ 42.58300018,  36.86700058,  11.01299953],
       [ 22.7310009 ,  54.88199997,  39.13800049],
       [ 69.51100159,  34.54999924,  27.00699997],
       [ 49.21099854,  27.49600029,  60.38100052],
       [ 49.60200119,  20.21199989,   6.79400015],
       [ 30.53499985,   4.55800009,   7.03599977],
       [  9.27799988,   8.9829998 ,  38.02500153],
       [  8.0369997 ,  66.09799957,  39.86299896],
       [ 55.25999832,   4.08699989,  67.40000153],
       [ 30.85899925,  59.57500076,  10.19099998],
       [ 26.04100037,  13.68000031,  44.57300186],
       [ 51.84799957,  22.13299942,  33.64899826],
       [  7.06699991,  25.31699944,  59.99300003],
       [ 23.29599953,  33.74100113,  74.61499786],
       [ 64.3690033 ,  68.68099976,  30.96899986],
       [ 18.94499969,  39.68700

In [3]:
start = time.time()
sc.parallelize(range(200), 4).count()
print "Count, %.2f"%((time.time()-start))

Count, 26.45


In [5]:
row_rdd.cartesian(row_rdd).map(lambda a: a).collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:703)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:702)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:79)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:702)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:1514)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:84)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:1438)
	at org.apache.spark.SparkContext$$anonfun$stop$7.apply$mcV$sp(SparkContext.scala:1724)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1185)
	at org.apache.spark.SparkContext.stop(SparkCon

In [ ]:
start = time.time()
distances=  row_rdd.cartesian(row_rdd).\
            map(lambda a: (a[0].squared_distance(a[1]))).\
            filter(lambda a: a>15.0).\
            saveAsTextFile("distances.csv")
print "ComputeDistance, %.2f"%(time.time()-start)

## n-D Partitioning

In [ ]:
num_partitions=4

def compute_distance_2d(partition_index):
    # 2-D Partitioning
    coord_all = coord_broadcast.value[:100]
    length = len(coord_all)
    # identify square to work on    
    xdim = math.sqrt(num_partitions)
    ydim = math.sqrt(num_partitions)
    xdim/partition_index
    len=len(coord_all)
    

## 2.4 Leaflet Finder Pilot-InMemory Implementation (Graph)

In [ ]:
from distributed_inmem.dataunit_spark import DistributedInMemoryDataUnit
import time

FILENAME="../data/mdanalysis/small/graph_edges_95_215.csv"

FILENAME_ALL_EDGES="../data/mdanalysis/small/graph_edges_95_215_alledges.csv"
du = DistributedInMemoryDataUnit(name="LeafletFinderGraph", sc=sc)

#DistributedInMemoryDataUnit.spark_context.version

f = open(FILENAME_ALL_EDGES)
graph = f.readlines()
du.load(graph)
f.close()

def identityMapper(edge, args):
    #print edge
    #comp = edge.strip().split(",")
    #return (int(comp[0]), int(comp[1]))
    return eval(str(edge))

def groupByVertex(data):
    print("Call reduce on: " + str(data))
    

new_iteration_needed = du.sc.accumulator(0)

# check for smaller keys in each set
def process_vertex(vertex):
    """ pass single vertex and its adjecent vertices
        e.g.: (0, [0, 67, 14])
    """
    global new_iteration_needed
    vertex = eval(vertex)
    source = int(vertex[0])
    dest= sorted([int(i) for i in vertex[1]])
    local_max = False
    
    first_edge_destination = int(dest[0])
    new_vertices = []    
    print "*********Source: %d First Edge Dest: %d"%(source, first_edge_destination) 
    if source <= first_edge_destination:
        local_max = True
        new_vertices.append((source, first_edge_destination))
            

    print "Process: " + str(vertex) + " Local Max: " + str(local_max)
    last_edge_destination = first_edge_destination

    for current_destination in vertex[1]:
        print "Current destination: %s"%str(current_destination)
        current_destination = int(current_destination)
        if current_destination == last_edge_destination: 
            continue
        
        if local_max == True:
            edge = (source, current_destination)
            new_vertices.append(edge)
        else:
            new_vertices.append((first_edge_destination, current_destination))
            new_vertices.append((current_destination, first_edge_destination))
            print "Add 1 to accumulator"
            new_iteration_needed.add(1)

        last_edge_destination = current_destination
    
    if ((not local_max) and (source < last_edge_destination)):
        new_vertices.append((source, first_edge_destination))
    
    print "Return new vertices: " + str(new_vertices)
    return new_vertices


#process_vertex("('19', ['19', '7', '9', '41'])")
num_iterations=0
start = time.time()
while True:
    old_accum_value = new_iteration_needed.value
    print "*********** Start iteration: %d " % num_iterations
    future_result = du.map_pilot(identityMapper, None, number_of_compute_units=2)
    result_du=future_result.result()[0]
    future_result = result_du.reduce_pilot(process_vertex, number_of_compute_units=2)
    output = future_result.result()
    output.export()   
    du = output
    num_iterations = num_iterations + 1
    print "New iteration accum: %d old value: %d"%(new_iteration_needed.value, old_accum_value)
    if old_accum_value < new_iteration_needed.value:
        #print "Accumulator value was increased. New iteration."
        continue        
        #pass
    else:
        break
    break
end = time.time()
print "Final results: "
num_components=du.data.groupByKey().count()
print "Finished after %d Iterations. Found %d components. Time: %.2f"%(num_iterations, num_components, (end-start)) 


In [ ]:
du.data.groupByKey().count()

## 2.5 Native Spark Implementation

In [3]:
!hadoop fs -ls /data/mdanalysis/large

Found 1 items
-rw-r--r--   3 luckow hdfs   29855041 2015-11-25 00:03 /data/mdanalysis/large/graph_edges_145746_1012872.csv


In [ ]:
rdd = sc.textFile("/data/mdanalysis/large/graph_edges_145746_1012872.csv")

### 2.5.1 Load data from text file 

In [8]:
%%time
FILENAME="/data/mdanalysis/small/graph_edges_95_215.csv"
data = sc.textFile(FILENAME).map(lambda line: [int(i) for i in line.split(",")])
# add backward edges
data = data.flatMap(lambda v: [(v[0],v[1]),(v[1],v[0])])

#data.saveAsTextFile("../data/mdanalysis/small/graph_edges_95_215_alledges.csv")
#data = data.filter(lambda v: v[0] != v[1])
#print data.collect()

data_grouped = data.groupByKey().mapValues(lambda a: sorted(set(a)))
print data_grouped.collect()

[(0, [0, 14, 67]), (2, [2, 13, 34, 62]), (4, [4, 27, 33, 68, 94]), (6, [6, 57, 64, 67]), (8, [8, 46, 69, 88, 93]), (10, [10, 31, 40, 71]), (12, [7, 12, 30, 50, 91]), (14, [0, 14, 48, 64]), (16, [16, 40]), (18, [18]), (20, [20, 85]), (22, [22, 54]), (24, [24, 62]), (26, [26, 49, 57, 70]), (28, [11, 17, 28, 69]), (30, [9, 12, 23, 30, 50]), (32, [32, 53]), (34, [2, 34]), (36, [36, 47, 84, 92]), (38, [9, 23, 38]), (40, [10, 16, 31, 40, 71]), (42, [42, 73]), (44, [44, 68]), (46, [8, 46]), (48, [14, 48, 49]), (50, [12, 23, 30, 50]), (52, [29, 52, 79, 89]), (54, [22, 54, 87]), (56, [56, 76]), (58, [58, 81]), (60, [17, 60, 69, 81]), (62, [2, 24, 25, 62, 79, 89]), (64, [6, 14, 57, 64, 67]), (66, [51, 66]), (68, [4, 44, 68, 72, 94]), (70, [26, 70, 79, 86]), (72, [68, 72]), (74, [43, 47, 53, 74, 82]), (76, [56, 76]), (78, [78, 92]), (80, [43, 80]), (82, [74, 77, 82]), (84, [36, 63, 84]), (86, [70, 79, 86]), (88, [8, 69, 88, 93]), (90, [11, 55, 75, 90]), (92, [36, 47, 78, 92]), (94, [3, 4, 33, 59,

#### 2.3.2 Connected Component Implementation

In [9]:
new_iteration_needed = sc.accumulator(0)
# check for smaller keys in each set
def process_vertex(vertex):
    """ pass single vertex and its adjecent vertices
        e.g.: (0, [0, 67, 14])
    """
    global new_iteration_needed
    source = vertex[0]
    local_max = False
    
    first_edge_destination = vertex[1][0]
    new_vertices = []    
    print "*********Source: %d First Edge Dest: %d"%(source, first_edge_destination) 
    if source <= first_edge_destination:
        local_max = True
        new_vertices.append((source, first_edge_destination))
            
    #pdb.set_trace()
    print "Process: " + str(vertex) + " Local Max: " + str(local_max)
    last_edge_destination = first_edge_destination

    #if vertex[1]==None or len(vertex[1])<=1:
    #    new_vertices.append((source, source))   
    for current_destination in vertex[1]:
        #print "Current destination: %s"%str(current_destination)
        if current_destination == last_edge_destination: 
            continue
        
        if local_max == True:
            edge = (source, current_destination)
            new_vertices.append(edge)
        else:
            new_vertices.append((first_edge_destination, current_destination))
            new_vertices.append((current_destination, first_edge_destination))
            print "Add 1 to accumulator"
            new_iteration_needed.add(1)

        last_edge_destination = current_destination
    
    if ((not local_max) and (source < last_edge_destination)):
        new_vertices.append((source, first_edge_destination))
    
    #print "Return new vertices: " + str(new_vertices)
    return new_vertices


#process_vertex((19, [7, 9, 19, 41]))
num_iterations=0
cc = data_grouped
start = time.time()
while True:
    old_accum_value = new_iteration_needed.value
    print "*********** Start iteration: %d " % num_iterations
    #print "Accum before iteration: " + str(old_accum_value)
    cc = cc.flatMap(lambda v: process_vertex(v))\
           .groupByKey()\
           .mapValues(lambda a: sorted(set(a)))
    cc.collect()
    num_iterations = num_iterations + 1
    #print "New iteration accum: %d old value: %d"%(new_iteration_needed.value, old_accum_value)
    if old_accum_value < new_iteration_needed.value:
        #print "Accumulator value was increased. New iteration."
        continue
    else:
        break
end = time.time()

print "Finished after %d Iterations. Found %d components. Time: %.2f"%(num_iterations, cc.count(), (end-start))

*********** Start iteration: 0 
*********** Start iteration: 1 
*********** Start iteration: 2 
*********** Start iteration: 3 
*********** Start iteration: 4 
*********** Start iteration: 5 
Finished after 6 Iterations. Found 10 components. Time: 2.39


## 3. Benchmark

---
## 4. Scratch Space

In [ ]:
#start = df_grouped
result=start.flatMap(lambda v: (v[0], v[1])).map(lambda v: v<start_index).countByValue()

print sttrresult

local_max = not result.has_key(True)

print "Local Max: " + str(local_max) + " Smaller Index: " + str(result.has_key(True))

In [ ]:
schema = StructType([
            StructField("source", IntegerType(), True),
            StructField("destination", IntegerType(), True)
        ])
df = sqlCtx.createDataFrame(data, schema)
df.explain()
schema_grouped = StructType([
            StructField("source", IntegerType(), True),
            StructField("destination", ArrayType(IntegerType()), True)
        ])
df_grouped = sqlCtx.createDataFrame(data_grouped, schema_grouped)

In [ ]:
from pyspark.sql.functions import udf, lit
from pyspark.sql.functions import *
from pyspark.sql.types import ArrayType

t = udf(lambda s: str(s), StringType())
slen = udf(lambda s: Column(len(s)), IntegerType())

#df.groupBy("source").collect()
#df.groupBy("source").agg(df.source, t(df.source))

c = df.groupBy(df.source).agg(col("source"), slen(df.destination))

#c = df.agg(col("source"), t(df.destination).alias('counts'))
c.head(5)


#c = df.groupBy(df.source).lit(df.destination)


In [ ]:
vertices = df.select(df["source"]).unionAll(df.select(df["destination"]))
vertices = di_source.distinct()

print "Number of vertices: %d"%(vertices.count())

# GraphLab

In [ ]:
from graphlab import SGraph, SFrame
from graphlab import connected_components

PROBLEM={"small": "./data/mdanalysis/small/graph_edges_95_215.csv",
         "medium":"./data/mdanalysis/medium/graph_edges_24056_71826.csv"}

d =datetime.datetime.now()
RESULTSFILE = "results-" + d.strftime("%Y%m%d-%H%M%S") + ".csv"
REPEATS=5

start = time.time()
data = SFrame.read_csv(filename, header=False)
sg = SGraph().add_edges(data, src_field="X1", dst_field="X2")
end_read=time.time()
cc = connected_components.create(sg)
s=cc["component_size"]
end_connected = time.time()
print cc
print s